In [58]:
import pandas as pd
import numpy as np
import re

# Building a spam filter for SMS messages

The goal of this project is to build a spam filter that classifies SMS messages as spams or hams (i.e., non-spam messages).

The data set we will use to train and test our classifier is available in the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

The [dataset](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) contains 5572 messages labelled as spam or non-spam (ham). 

We'll build the spam filter using the multinomial Naive Bayes Algorithm, and our goal is to reach an accuracy > 95%.

In this project we'll do the following:
- Split the dataset into a training set and a test set
- Create a vocabulary of the unique words of all the SMS messages in our dataset
- Convert the SMS messages into a word frequency matrix
- Calculate the constants and parameters of the Naive Bayes Algorithm
- Build the spam filter
- Measure the accuracy of the spam filter

The multinomial Naive Bayes algorithm we'll implement is the following:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}
where $P(Spam | w_1,w_2, ..., w_n)$ is the probability a message is a spam, given its words, and $P(w_i|Spam)$ is the probability a word occurs in the message, given it is a spam. 

The same reasoning is then applied to estimate the probability a message is not a spam.

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}


In [59]:
# Reading the dataset
df=pd.read_table("https://raw.githubusercontent.com/Artur-Wanderley/Portfolio-of-my-data-science-projects/master/Building%20a%20Spam%20Filter%20with%20Naive%20Bayes/SMSSpamCollection",sep="\t", header=None,names=["label","SMS"])
print(df.shape)
df.head()

(5572, 2)


,label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# Looking at the dataset

In [60]:
# How many rows and columns the dataset have?
df.shape

(5572, 2)

In [61]:
# Looking for NaNs
df.isnull().sum()

label    0
SMS      0
dtype: int64

In [62]:
# Proportion of hams vs spams
print("Percentage of non-spam and spam messages:\n",df["label"].value_counts(normalize=True)*100)

Percentage of non-spam and spam messages:
 ham     86.593683
spam    13.406317
Name: label, dtype: float64


As we can see above, about 87% of the messages are non-spam (hams), whereas about 13% of the messages are spams.

# Training and test set

Before we start building the spam filter, let's split our dataset into two sets, a training set, and a test set. With the training set, we'll build the spam filter, and with the test set, we'll evaluate how good the spam filter is to classify SMS messages as spams.

We'll use about 80% of the dataset as the training set (4458 messages) and about 20% of the dataset as the test set (1114 messages). 

In [63]:
# Splitting the data into a training and a test set

# Randomizing the dataset before splitting it 
random_df=df.sample(frac=1,random_state=1) 

# Creating the index to split the dataset
split_index=round(len(random_df)*0.8)

# Splitting the randomized dataset into training and test set
training_set=random_df[:split_index].reset_index(drop=True)
test_set=random_df[split_index:].reset_index(drop=True)

# Checking the length of training and test set
print(len(training_set))
print(len(test_set))

4458
1114


Now, let's check whether the percentage of non-spam (87%) vs spam (13%) we observed in the original dataset remains in the training and test set.

In [64]:
# Percentages of non-spam vs spam in the training set
print(training_set["label"].value_counts(normalize=True))

ham     0.86541
spam    0.13459
Name: label, dtype: float64


In [65]:
# Percentages of non-spam vs spam in the test set
print(test_set["label"].value_counts(normalize=True))

ham     0.868043
spam    0.131957
Name: label, dtype: float64


As we can see above, the percentages of non-spam and spam messages remained practically the same in both the training and test set.

Now, let's move on and do some data cleaning.

 # Data cleaning
 ## Remove punctuation and letter case
 
To calculate the probabilities required by the Naive Bayes Algorithm, we'll remove the punctuation and letter case of the messages and convert the messages into lists of strings. This will facilitate extract the data to make it available for the algorithm.

Here is how we are going to transform the data:

![img](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_3.png)

We are going to convert the training set into a matrix of word frequency where each line is a message and the columns are the frequency of each word present in the SMS messages.

Let's start by removing punctuation and letter case from the SMS messages.

In [66]:
# Removing punctuation and letter case from the training set
training_set["SMS"]=training_set["SMS"].str.replace("\W"," ")
training_set["SMS"]=training_set["SMS"].str.lower()
training_set.head()

,label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


# Creating a vocabulary of the SMS messages

Remember from the top that the multinomial Naive Bayes Algorithm we're implementing requires the probability each word occurs in a spam message ($P(w_i|Spam)$). To calculate this probability, we'll need to create a vocabulary of all unique  words of the SMS messages and count them because:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation},

where $N_{Vocabulary}$ is the number of unique words in all messages (both spam and ham).

$N_{Vocabulary}$ is also required to calculate $P(w_i|Ham)$ because:

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Therefore, let's create the vocabulary of unique words found in the SMS messages.

In [67]:
# Transforming each SMS  message into a list of strings
training_set["SMS"]=training_set["SMS"].str.split()

# Creating the vocabulary
vocabulary=[]

for message in training_set["SMS"]:
    for word in message:
        vocabulary.append(word)
    
# Removing duplicates from the vocabulary
vocabulary=set(vocabulary) # This code remove duplicated words
vocabulary=list(vocabulary)

#Let's check how many words the vocabulary have
print("Number of unique words in the vocabulary:\n",len(vocabulary))

Number of unique words in the vocabulary:
 7783


# Converting the SMS messages into a word frequency matrix

Now that we have our vocabulary of 7783 unique words, let's convert our training set into a word-frequency matrix, as shown in the scheme above.

Remember that in our matrix of word frequency, each line is a message and each column is a unique word of the vocabulary.

In [68]:
# Creating a dictionary of word counts per SMS
word_counts_per_sms={unique_word:[0]*len(training_set["SMS"]) for unique_word in vocabulary}

# Counting the frequency of each word per SMS
for index,sms in enumerate(training_set["SMS"]):
    for word in sms:
        word_counts_per_sms[word][index]+=1

In [69]:
# Transforming the word counts dictionary into a dataset
word_counts_per_sms=pd.DataFrame(word_counts_per_sms)
word_counts_per_sms.head()

,girl,jerk,stress,gudnite,subscriber,apes,century,probpop,reached,ecstasy,...,carpark,dept,straight,cat,snowboarding,oclock,06,fiend,pansy,msg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
# Concatenating the original training set to the word frequency matrix
clean_training_set=pd.concat([training_set,word_counts_per_sms],axis=1)
clean_training_set.head()

,label,SMS,girl,jerk,stress,gudnite,subscriber,apes,century,probpop,...,carpark,dept,straight,cat,snowboarding,oclock,06,fiend,pansy,msg
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Calculating the constants of the Naive Bayes Algorithm

Now that our training set is ready, let's start building the Multinomial Naive Bayes Classifier.

We'll start by calculating the constants $P(Spam)$, $P(ham)$, $N_{spam}$,$N_{Ham}$, and $N_{Vocabulary}$.

Remember that the multinomial Naive Bayes Algorithm we are implementing to build the spam filter is the following.

Probability of spam:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Probability of ham:

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [71]:
# Calculating P(Ham) and P(Spam)
p_ham=len(clean_training_set[clean_training_set["label"]=="ham"])/clean_training_set.shape[0]
p_spam=len(clean_training_set[clean_training_set["label"]=="spam"])/clean_training_set.shape[0]

In [72]:
# Calculating N_ham
cols=clean_training_set.iloc[:,2:].columns.tolist()
n_ham=clean_training_set[clean_training_set["label"]=="ham"][cols].values.sum()
print("Number of words in ham messages:",n_ham)

# Calculating N_spam
cols=clean_training_set.iloc[:,2:].columns.tolist()
n_spam=clean_training_set[clean_training_set["label"]=="spam"][cols].values.sum()
print("Number of words in spam messages:",n_spam)

# n_vocabulary

n_vocabulary=len(vocabulary)

# Creating the alpha variable for Laplace smoothing
alpha=1

Number of words in ham messages: 57237
Number of words in spam messages: 15190


# Calculating the parameters

After calculating the constants of the Naive Bayes Algorithm, let's calculate the parameters $P(w_{i}|Spam)$ and $P(w_{i}|Ham)$.

In [73]:
# Calculating p_word_given_ham and p_word_given_spam
p_ham_dic={unique_word:0 for unique_word in vocabulary}
p_spam_dic={unique_word:0 for unique_word in vocabulary}
ham_sms=clean_training_set[clean_training_set["label"]=="ham"]
spam_sms=clean_training_set[clean_training_set["label"]=="spam"]

for word in vocabulary:
    n_word_given_ham=ham_sms[word].sum()
    p_word_given_ham=(n_word_given_ham+alpha)/(n_ham+alpha*n_vocabulary)
    p_ham_dic[word]=p_word_given_ham
    
    n_word_given_spam=spam_sms[word].sum()
    p_word_given_spam=(n_word_given_spam+alpha)/(n_spam+alpha*n_vocabulary)
    p_spam_dic[word]=p_word_given_spam

# Building the spam filter

Now that we have calculated all the constants and parameters, it's time to create our spam filter.

The spam filter we'll create does the following:
- Takes in a new message
- Calculates the probability this new message is a ham ($P(Ham | w_1,w_2, ..., w_n)$) and the probability it is a spam ($P(Spam | w_1,w_2, ..., w_n)$)
 - If $P(Ham | w_1,w_2, ..., w_n) > P(Spam | w_1,w_2, ..., w_n)$, the message is classified as ham
 - If $P(Ham | w_1,w_2, ..., w_n) < P(Spam | w_1,w_2, ..., w_n)$, the message is classified as spam
 - If $P(Ham | w_1,w_2, ..., w_n)$ $=$ $P(Ham | w_1,w_2, ..., w_n)$, the algorithm will request human help to classify the message

In [74]:
# Building the spam filter
def classify(message):
    message=re.sub("\W"," ",message) # Removes message punctuation
    message=message.lower().split()
    
    p_spam_given_message=p_spam
    p_ham_given_message=p_ham
    
    for word in message:
        if word in p_spam_dic:
            p_spam_given_message*=p_spam_dic[word]
                   
        if word in p_ham_dic:
            p_ham_given_message*=p_ham_dic[word]         
    
    print("P(spam|message):",p_spam_given_message)
    print("P(ham|message):",p_ham_given_message)
    
  
    if p_spam_given_message > p_ham_given_message:
        print("label: spam")
    
    elif p_spam_given_message < p_ham_given_message:
        print("label: ham")
    
    else:
        print('Equal proabilities, have a human classify this!')

## Testing the spam filter

Let's try our spam filter!

In [75]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(spam|message): 1.3481290211300841e-25
P(ham|message): 1.9368049028589875e-27
label: spam


In [76]:
classify("Sounds good, Tom, then see u there")

P(spam|message): 2.4372375665888117e-25
P(ham|message): 3.687530435009238e-21
label: ham


# Measuring the spam filter accuracy

Let's use the test set to measure how accurate our spam filter is.

However, to test our spam filter, let's do some editing in the `classify` function.

In [77]:
# Editing the spam filter code
def classify(message):
    message=re.sub("\W"," ",message)
    message=message.lower().split()
    
    p_spam_given_message=p_spam
    p_ham_given_message=p_ham
    
    for word in message:
        if word in p_spam_dic:
            p_spam_given_message*=p_spam_dic[word]
                   
        if word in p_ham_dic:
            p_ham_given_message*=p_ham_dic[word]         
    
#     print("P(spam|message):",p_spam_given_message)
#     print("P(ham|message):",p_ham_given_message)
    
  
    if p_spam_given_message > p_ham_given_message:
        return "spam"
    
    elif p_spam_given_message < p_ham_given_message:
        return "ham"
    else:
        return "needs human classification"

In [78]:
# Measuring the accuracy of our spam filter
test_set["predicted"]=test_set["SMS"].apply(classify)

correct_predictions=0

for i in range(len(test_set)):
    if test_set.loc[i,"label"]==test_set.loc[i,"predicted"]:
        correct_predictions+=1
        
print("Correct:",correct_predictions)
print("Incorrect:", len(test_set)-correct_predictions)
print("Accuracy:",round(correct_predictions/len(test_set),3))
       

Correct: 1100
Incorrect: 14
Accuracy: 0.987


# The spam filter is nearly 99% accurate!

As we can see above, of the 1114 messages in our test set, the spam correctly assigned 1000 SMS messages as spam ou ham.

This is a pretty good performance!